In [1]:
import numpy as np
import pandas as pd

In [4]:
temp_df=pd.read_csv(r'C:\Users\Abdur rahim nishad\Downloads\archive (3)\IMDB Dataset.csv')

In [5]:
temp_df.iloc[:1000]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [6]:
df=temp_df.loc[:1000]

In [7]:
df.duplicated().sum()

0

In [8]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile("<.*?>"), "",raw_text)
    return cleaned_text

In [9]:
df["review"]=df.review.apply(remove_tags)

C:\Users\ABDURR~1\AppData\Local\Temp/ipykernel_9896/1438303900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"]=df.review.apply(remove_tags)


In [10]:
df.review=df.review.str.lower()

C:\Users\Abdur rahim nishad\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
from nltk.corpus import stopwords
sw_list=stopwords.words("english")

C:\Users\Abdur rahim nishad\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
def remove_stopwords(text):
    text=text.split()
    li=[]
    
    for i in text:
        if i not in stopwords.words("english"):
            li.append(i)
    return " ".join(li)

In [13]:
remove_stopwords("Hi I am Riad")

'Hi I Riad'

In [14]:
df.review=df.review.apply(remove_stopwords)

C:\Users\Abdur rahim nishad\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
from nltk.tokenize import word_tokenize,sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
story=[]
for doc in df["review"]:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [23]:
model=gensim.models.Word2Vec(
        window=10,
        min_count=2
)

<IPython.core.display.Javascript object>

In [24]:
model.build_vocab(story)

In [25]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(557540, 623840)

In [26]:
len(model.wv.index_to_key)

9397

In [30]:
def document_vector(doc):
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [31]:
document_vector(df["review"].values[0])

array([-0.22930428,  0.35537463,  0.16608283,  0.03547059,  0.04191952,
       -0.5558211 ,  0.20883194,  0.75902724, -0.37820303, -0.24127206,
       -0.11698265, -0.575418  , -0.14104404,  0.19496764,  0.04454205,
       -0.25106764,  0.04834866, -0.5117949 ,  0.05720061, -0.77706844,
        0.21577497,  0.19170576,  0.34522977, -0.2196229 ,  0.1002078 ,
        0.05622682, -0.2951289 , -0.1754264 , -0.44746116, -0.035493  ,
        0.33139002,  0.18777815,  0.04640797, -0.16386528, -0.17369503,
        0.390661  ,  0.14350426, -0.09106061, -0.18787953, -0.6564527 ,
       -0.00121933, -0.2759961 , -0.14900811,  0.02537924,  0.31026167,
       -0.18168154, -0.3399149 ,  0.01252251,  0.29763559,  0.34229013,
        0.1756602 , -0.4125791 , -0.26707587, -0.12038393, -0.1289344 ,
        0.27736443,  0.23449723, -0.03960999, -0.2878791 ,  0.21433331,
        0.1099663 ,  0.21689089, -0.08817928, -0.06171646, -0.52054775,
        0.42043376,  0.09068713,  0.1681286 , -0.40051916,  0.42

In [32]:
from tqdm import tqdm

In [33]:
X=[]
for doc in tqdm(df["review"].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [00:22<00:00, 43.94it/s]


In [34]:
X=np.array(X)

In [35]:
X.shape

(1001, 100)

In [36]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(df["sentiment"])

In [37]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=1)

In [39]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [40]:
mnb=GaussianNB()
mnb.fit(X_train,y_train)
y_pred=mnb.predict(X_test)
accuracy_score(y_test,y_pred)

0.5323383084577115

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.6119402985074627